# JijModeling 1.8.0 リリースノート

In [1]:
import jijmodeling as jm
import ommx.v1
import pytest

## Compiling into OMMX Message

![image.png](./assets/jijmodeling-1.8.0_01.png)

- `jijmodeling` 単体でモデルへのコンパイルが行えるようになりました
  - [`jijmodeling_transpiler`](https://pypi.org/project/jijmodeling-transpiler/)の利用は必須ではなくなりました！

### `Interpreter.eval_scalar`

- `eval_scalar` は決定変数を含まないJijModeling式を浮動小数点数値へと評価します。
- テストやデバッグ用途に便利です。

In [ ]:
# インスタンスデータによる初期化
interpreter = jm.Interpreter({"a": [1, 2, 3]})

# プレースホルダはインスタンスデータにより評価される
a = jm.Placeholder("a", ndim=1)
assert interpreter.eval_scalar(a[1]) == 2.0

In [ ]:
# 総和記号もサポート済
n = a.len_at(0, latex="n")
i = jm.Element("i", belong_to=n)
sum_a = jm.sum(i, a[i])
sum_a

sum(Element(name='i', belong_to=(NumberLit(value=0), ArrayLength(array=Placeholder(name='a', ndim=1), axis=0, latex='n'))), Placeholder(name='a', ndim=1)[Element(name='i', belong_to=(NumberLit(value=0), ArrayLength(array=Placeholder(name='a', ndim=1), axis=0, latex='n')))])

In [4]:
assert interpreter.eval_scalar(sum_a) == 1 + 2 + 3

In [ ]:
# 式が決定変数を含む場合は実行時エラー
v = jm.BinaryVar("v")
with pytest.raises(jm.InterpreterError):
    interpreter.eval_scalar(v)

### `Interpreter.eval_expr`

決定変数を含む JijModeling 式は、`eval_expr` により `ommx.v1.Function` へと評価できます。

In [ ]:
# 決定変数群
x = jm.BinaryVar("x", shape=(n,))

# 線型関数
f = jm.sum(i, a[i] * x[i])
f

sum(Element(name='i', belong_to=(NumberLit(value=0), ArrayLength(array=Placeholder(name='a', ndim=1), axis=0, latex='n'))), Placeholder(name='a', ndim=1)[Element(name='i', belong_to=(NumberLit(value=0), ArrayLength(array=Placeholder(name='a', ndim=1), axis=0, latex='n')))] * BinaryVar(name='x', shape=[ArrayLength(array=Placeholder(name='a', ndim=1), axis=0, latex='n')])[Element(name='i', belong_to=(NumberLit(value=0), ArrayLength(array=Placeholder(name='a', ndim=1), axis=0, latex='n')))])

In [7]:
f_ = interpreter.eval_expr(f)
assert type(f_) is ommx.v1.Function
f_

Function(x0 + 2*x1 + 3*x2)

- 決定変数は評価時に Interpreter へと登録されます。
- 決定変数のIDは、`Interpreter` への登録時に 0 から連続的に付番されます。
  - ここでは `x[0]` の ID は 0、`x[1]` は 1、……となります。
- `ommx.v1.Function` には元の変数名の情報が含まれていないため、Python の `__repr__` 関数により表示すると、変数はIDによって `x0`, `x_1`, ... と表示されることに注意してください。

In [ ]:
y = jm.BinaryVar("y")
interpreter.eval_expr(y)  # 変数名 `y` ではなく、ID により `x3` と表示される。

Function(x3)

In [ ]:
# OMMX側の対応する決定変数を取得
x0 = interpreter.get_decision_variable_by_name("x", [0])
x1 = interpreter.get_decision_variable_by_name("x", [1])
x2 = interpreter.get_decision_variable_by_name("x", [2])
assert type(x0) is ommx.v1.DecisionVariable

# 取得した ommx.v1.Function を比較
assert f_.almost_equal(ommx.v1.Function(x0 + 2*x1 + 3 * x2))

### `Interpreter.eval_problem`

In [10]:
problem = jm.Problem("release-1.8.0")
problem += f
problem += jm.Constraint("quad", jm.sum(i, x[i] * x[i]) == 1)
problem

In [11]:
instance = interpreter.eval_problem(problem)
assert type(instance) is ommx.v1.Instance

### 制限

- `jijmodeling.CustomPenaltyTerm` は未対応です。
  - 次リリースでOMMXの更新と共にサポート予定です。